# Chatbot model of Virtual Clinic

In [2]:
import json

In [5]:
# Opening JSON file
f = open('intent_medical.json')

# returns JSON object as
# a dictionary
data = json.load(f)
data

{'intents': [{'tag': 'greeting',
   'patterns': ['Hi', 'Hey', 'Is anyone there?', 'Hello', 'Hay'],
   'responses': ['Hello', 'Hi', 'Hi there']},
  {'tag': 'goodbye',
   'patterns': ['Bye', 'See you later', 'Goodbye'],
   'responses': ['See you later', 'Have a nice day', 'Bye! Come back again']},
  {'tag': 'thanks',
   'patterns': ['Thanks',
    'Thank you',
    "That's helpful",
    'Thanks for the help'],
   'responses': ['Happy to help!',
    'Any time!',
    'My pleasure',
    "You're most welcome!"]},
  {'tag': 'fever',
   'patterns': [' I am having 101 fever, can I take paracetamol?',
    'I am having fever',
    'I am not feeling well'],
   'responses': ['Yes, you can take paracetamol.',
    'Yes, you feel better, if you take paracetamol ',
    'If you dont have allergies then you          can take paracetamol.',
    'Once visit to doctor']},
  {'tag': 'pain',
   'patterns': ['I have neck pain, is it a sign of COVID?',
    ' I am feeling breathlessness',
    'Are you suffering fr

In [6]:
import json 
import numpy as np 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

In [9]:
with open('intent_medical.json') as file:
    data = json.load(file)
    
training_sentences = []
training_labels = []
labels = []
responses = []


for intent in data['intents']:
    for pattern in intent['patterns']:
        training_sentences.append(pattern)
        training_labels.append(intent['tag'])
    responses.append(intent['responses'])
    
    if intent['tag'] not in labels:
        labels.append(intent['tag'])
        
num_classes = len(labels)

In [10]:
lbl_encoder = LabelEncoder()
lbl_encoder.fit(training_labels)
training_labels = lbl_encoder.transform(training_labels)

In [11]:
vocab_size = 1000
embedding_dim = 16
max_len = 20
oov_token = "<OOV>"

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded_sequences = pad_sequences(sequences, truncating='post', maxlen=max_len)

## Model Training

In [12]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(GlobalAveragePooling1D())
model.add(Dense(16, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', 
              optimizer='adam', metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 16)            16000     
                                                                 
 global_average_pooling1d (G  (None, 16)               0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 16)                272       
                                                                 
 dense_1 (Dense)             (None, 16)                272       
                                                                 
 dense_2 (Dense)             (None, 8)                 136       
                                                                 
Total params: 16,680
Trainable params: 16,680
Non-trainable params: 0
____________________________________________________

In [13]:
epochs = 500
history = model.fit(padded_sequences, np.array(training_labels), epochs=epochs)

Epoch 1/500
1/1 [==============================] - 1s 1s/step - loss: 2.0804 - accuracy: 0.0370
Epoch 2/500
1/1 [==============================] - 0s 8ms/step - loss: 2.0787 - accuracy: 0.0741
Epoch 3/500
1/1 [==============================] - 0s 6ms/step - loss: 2.0773 - accuracy: 0.0741
Epoch 4/500
1/1 [==============================] - 0s 6ms/step - loss: 2.0759 - accuracy: 0.1481
Epoch 5/500
1/1 [==============================] - 0s 8ms/step - loss: 2.0748 - accuracy: 0.1481
Epoch 6/500
1/1 [==============================] - 0s 9ms/step - loss: 2.0737 - accuracy: 0.1481
Epoch 7/500
1/1 [==============================] - 0s 5ms/step - loss: 2.0727 - accuracy: 0.1481
Epoch 8/500
1/1 [==============================] - 0s 11ms/step - loss: 2.0715 - accuracy: 0.1481
Epoch 9/500
1/1 [==============================] - 0s 7ms/step - loss: 2.0704 - accuracy: 0.1481
Epoch 10/500
1/1 [==============================] - 0s 8ms/step - loss: 2.0693 - accuracy: 0.1481
Epoch 11/500
1/1 [===========

In [14]:
# to save the trained model
model.save("chat_model")

import pickle

# to save the fitted tokenizer
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
# to save the fitted label encoder
with open('label_encoder.pickle', 'wb') as ecn_file:
    pickle.dump(lbl_encoder, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)

INFO:tensorflow:Assets written to: chat_model\assets


In [16]:
import json 
import numpy as np
from tensorflow import keras
from sklearn.preprocessing import LabelEncoder

import colorama 
colorama.init()
from colorama import Fore, Style, Back

import random
import pickle

with open("intent_medical.json") as file:
    data = json.load(file)


def chat():
    # load trained model
    model = keras.models.load_model('chat_model')

    # load tokenizer object
    with open('tokenizer.pickle', 'rb') as handle:
        tokenizer = pickle.load(handle)

    # load label encoder object
    with open('label_encoder.pickle', 'rb') as enc:
        lbl_encoder = pickle.load(enc)

    # parameters
    max_len = 20
    
    while True:
        print(Fore.LIGHTBLUE_EX + "User: " + Style.RESET_ALL, end="")
        inp = input()
        if inp.lower() == "quit":
            break

        result = model.predict(keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences([inp]),
                                             truncating='post', maxlen=max_len))
        tag = lbl_encoder.inverse_transform([np.argmax(result)])

        for i in data['intents']:
            if i['tag'] == tag:
                print(Fore.GREEN + "ChatBot:" + Style.RESET_ALL , np.random.choice(i['responses']))

        # print(Fore.GREEN + "ChatBot:" + Style.RESET_ALL,random.choice(responses))

print(Fore.YELLOW + "Start messaging with the bot (type quit to stop)!" + Style.RESET_ALL)
chat()

Start messaging with the bot (type quit to stop)!
User: hi
ChatBot: Hi
User: I am not feeling well
ChatBot: Yes, you can take paracetamol.
User: I am having 101 fever
ChatBot: Yes, you feel better, if you take paracetamol 
User: I have neck pain
ChatBot: Tell me how can assist you
User: I have neck pain
ChatBot: Tell me your problem to assist you
User: I have neck pain
ChatBot: Yes Sure, How can I support you
User: I have neck pain
ChatBot: Tell me your problem to assist you
User: I have neck pain, is it a sign of COVID?
ChatBot: Yes.it is a sign of COVID.
User:  I am feeling weak, what to do?
ChatBot: It is better to visit doctor
User:  I am feeling weak, what to do?
ChatBot: It might be a symptom of COVID-19
User: My grandmother is diabetic
ChatBot: Hello
User: My grandmother is diabetic, and recently tested positive, what medicine to take?
ChatBot: Before take any medicine, It is better to visit doctor
User: My grandmother is diabetic, and recently tested positive, what medicine to 